# A. Input data-sets

pyPARAGON requires two inputs: 

    i) A reference network which is composed of already known interactions, 
    ii) Seed nodes which can come from in omics datasets, or be frequent mutations in disases. 


### Preparing a reference network

A reference network is introduced to pyPARAGON as a networkx object. Reference networks can include nan-nan interactions, causing troubles in calculations. Thus, we exclude nan-nan interactions in reference networks. Additionally, pyPARAGON does not consider self-interactions in graphlets. Therefore, we also eliminate self interactions to get rid of computational problems. Except for nan-nan and self-interactions, we may not use any filtration or additional preprocessing steps. pyPARAGON is able to eliminate the interactions with low confidence scores thanks to the flux calculation. Thus, we suggest not applying a confidence threshold value for the filtration. 

In [1]:
import pandas as pd, networkx as nx

In [2]:
HIPPIE_df=pd.read_csv("../Source/Interactomes/HIPPIE_v2_3.tab", keep_default_na=False, sep="\t") # excluding rows with nan value
HIPPIE_df

,Gene_1,Gene_2,Score
0,ALDH1A1,ALDH1A1,0.76
1,ITGA7,CHRNA1,0.73
2,PPP1R9A,ACTG1,0.65
3,SRGN,CD44,0.63
4,GRB7,ERBB2,0.90
...,...,...,...
783177,STIL,CDK5RAP2,0.63
783178,SIRT7,SNORA10,0.63
783179,BMP2,SNX18,0.63
783180,C1QBP,MRPL50,0.63


In [3]:

# Networkx object and removing self interactions
HIPPIE_nx=nx.from_pandas_edgelist(HIPPIE_df,"Gene_1","Gene_2",edge_attr="Score")
self_interactions=list(nx.selfloop_edges(HIPPIE_nx))


for u, v in self_interactions:
    if isinstance(u, float):
        print(u)
    else:
        HIPPIE_nx.remove_edge(u,v)
    
HIPPIE_df=nx.to_pandas_edgelist(HIPPIE_nx)
print("The number of edges in HIPPIE network",HIPPIE_nx.number_of_edges())
print("The number of nodes in HIPPIE network",HIPPIE_nx.number_of_nodes())


edges=tuple(zip(HIPPIE_df.source,HIPPIE_df.target))
for edge in edges:
    if edge not in HIPPIE_nx.edges:
        print("check the edge",edge)

The number of edges in HIPPIE network 774448
The number of nodes in HIPPIE network 19437


###  Preparing seed nodes

Seed nodes are known or detected context-specific knowledge. They can come from omics datasets, multi-omics datasets, or well-known, frequent mutations in diseases. Score your seeds as 1 if you do not generate initial node weights from raw data.

During multi-omics data integration, the user can follow two strategies with pyPARAGON. In the first strategy,  your hits in omics datasets can be transformed into nodes represented in the reference network. Identifying possible transcription factors which target differential hits in transcriptomics or bind to the genome can be useful to carry transcriptomics and genomics knowledge into the interactomes.  With the second use option of pyPARAGON in multi-omic datasets, you may extend the reference network by merging various types of networks, such as regulator networks, interactomes, metabolic networks, etc. In this case, you can directly use multi-omics hits, which are represented in the reference network. 

In [4]:
initial_nodes_df=pd.read_csv(f'../Source/Netpath/Sampling_0_5/AndrogenReceptor_05A/AndrogenReceptor_05A_var_0.nodes',sep="\t")

initial_nodes=initial_nodes_df.name.to_list()
initial_weights=[1 for _ in initial_nodes]

## B) Constructing Graphlet Guided Network 

During the construction of a graphlet-guided network (GGN), pyPARAGON does not consider the confidence scores of edges in parsing the graphlets. The advanced usage of pyPARAGON in the construction of GGN is detailed in the Graphlet_Guided_Network_Construction, covering reference network permutations, graphlet frequencies, and graphlet motif selections. In this script, we recruit four-node graphlets, such as Graphlet5, Graphlet6, Graphlet7, and Graphlet8. During graphlet generation, we derived four-node graphlets from three-node graphlets. Also you can directly select four-node graphlets in pyPARAGON to overcome the computational cost. 

In [5]:
from Paragon import GraphletGuidance

In [6]:
Graphlet_list=['Graphlets5', 'Graphlets6', 'Graphlets7', 'Graphlets8']

In [7]:

GRF_lite=GraphletGuidance(HIPPIE_nx)

In [8]:
GGN_nx=GRF_lite.construct_GGN(initial_nodes,Graphlets=Graphlet_list)

82 of 82 input nodes have been found in the given network



In [9]:
GGN_nx.number_of_nodes()

4137

In [10]:
GGN_df=nx.to_pandas_edgelist(GGN_nx)
GGN_df

,source,target
0,CEBPA,SMARCA2
1,CEBPA,ESR1
2,CEBPA,SMAD3
3,CEBPA,CDK4
4,CEBPA,ATF2
...,...,...
20805,SPDEF,SLC30A9
20806,SPDEF,CEP57
20807,SPDEF,KLHL9
20808,SPDEF,SLC4A2


In [11]:
GRF_lite.write_guided_graphlet_network(f'../Outputs/AndrogenReceptor_GGN')

True

## C) PageRank flux and context-specific network inference

pyPARAGON, before infering a context-specific network, applies personalized PageRank algorithm and then calculates flux scores of edges. Among edges in the GGN, the union of highly scored edges constructs the context-specific network. In the application of personalized PageRank, the confidence scores and the weights of seed nodes affects the node propagation. When confidence scores or weights of edges in reference networks are not employed, pyPARAGON sets a default edge score of 1 to all edges. Similarly, if seed nodes are not weighted, pyPARAGON assigned default value, 1, to all seed nodes. 

Here, as a guided network, we applied the GGN, which is a trimmed form of reference network. We focus on the related region of reference networks. Reference networks, such as interactomes or regulatomes, are mainly integrated databases that cover a variety of databases without regard for context specificity. Thus, GGN selects the associated region of the reference network. However, at that point, the user may use tissue-specific, cell lineage-specific, or other guided networks as a guide, regardless of the GGN construction. Another case is that reference networks, such as yeast interactomes, may not be as highly connected as human reference networks. In this case, the user can use the reference network directly without a guide network. The PageRank Flux and Network Inference script details the use of guide networks.



In [12]:
from Paragon import NetworkInference 

In [13]:
pgrf=NetworkInference(network=HIPPIE_nx,guide_network=GGN_nx,edge_attribute="Score")

When invoking the network inference step in pyPARAGON, it is mandatory to introduce a reference network. pyPARAGON assumes the whole reference network functions as a guide network in the absence of a specific guide network introduction. If the user does not provide an edge attribute as a confidence score or another edge score, pyPARAGON assigns a default value of 1 to all edges. 

In [14]:
pgrf.load_initial_nodes(list(initial_nodes))

Seed nodes in this script lack weights. Thus, pyPARAGON gives a uniform value of 1 to all seed nodes. The user may input the specified weights of seed nodes ranging from 0 to 1, as explained in the PageRank Flux and Network Inference script.   

In [15]:
Infered_nx=pgrf.reconstruct_subnetwork( max_edge_count=2000,alpha=0.8,threshold=0.8)


theshold of 0.560000 limits predictions to 2000 edges


In [16]:
pgrf.write_created_network(f'../Outputs/AndrogenReceptor_inferred_context_specific_network')

True

# D) Interpreting a context specific network

pyPARAGON partitions a context-specific network into separate communities known as network modules for biological analysis, integrating these communities with biological annotations. This script explores over-representation analysis using gene ontology annotations. The user may expand the use of overrepresentation analysis to additional datasets like KEGG, Reactome, and others. The script of Interpretable_Communities_in_Subnetworks provides a full explanation of the interpretation module, community analysis.

In [17]:
from Paragon import CommunityAnalysis 

In [18]:
CA=CommunityAnalysis(Infered_nx)

### Community detection

In [19]:
module_df, node_df=CA.get_communities_in_DataFrames()
node_df

,Genes,Community
0,RNF14,Module_1
1,UBE2E3,Module_1
2,PHF7,Module_1
3,DET1,Module_1
4,LEF1,Module_1
...,...,...
352,UBD,Module_33
353,RAC3,Module_33
354,CDC42,Module_33
355,ARRB2,Module_33


In [20]:
node_df.to_csv(f'../Outputs/AndrogenReceptor_inferred_csn_nodes_in_communities.tab',sep="\t", index=False)

In [21]:
module_df

,Community_name,Community
0,Module_1,RNF14;UBE2E3;PHF7;DET1;LEF1
1,Module_2,DISC1;SFPQ;REL;UBE2D3;GPS2;TRIM68
2,Module_3,ZMYND10;CALM1;UBE2I;HIC1;TCF4;OGT;NEUROD1
3,Module_4,GRAP2;STK40;NXF1;ETV5;COP1;CHD3
4,Module_5,PRDM11;ERCC3;GTF2F1;GTF2H1;E2F1;PPARA;FGL1;PIK...
5,Module_6,TLE1;GTF2H2C;IL6R;CMYA5;AR;NR2C1;IL6ST;ERBB2;P...
6,Module_7,GTF2H2;CTD;SHLD2;GTF2H3;CCNH;MKI67;GTF2E1;ERCC...
7,Module_8,GRB2;NTRK1;CIT;MAPT;GSK3B;ACTB;HSP90AB1;CDK5;F...
8,Module_9,RAN;SUMO1;HSPA1A;UBE3A;PXN;EP300;GAPDH;HSPA8;C...
9,Module_10,ESR1;BRCA1;NR3C1;ARNT;AKT1;TP53;HSP90AA1;HSPB1...


In [22]:
module_df.to_csv(f'../Outputs/AndrogenReceptor_inferred_csn_communities.tab',sep="\t", index=False)

### Overrepresentation analysis (ORA) with gene ontology annotations

Reference knowledge for ORA is tabulated in two columns: Specific ID or name of annotations, and components which are represented in networks.   

In [23]:
GOA_bio_proc_df=pd.read_csv(f'../Source/Annotations/GOA_proteins_isoforms_prepared.tab',sep="\t")
GOA_bio_proc_df

,GO ID,DB Object Symbol
0,GO:0002250,IGKV3-7
1,GO:0002250,IGKV1D-42
2,GO:0002250,IGLV4-69
3,GO:0002250,IGLV8-61
4,GO:0002250,IGLV4-60
...,...,...
210592,GO:0006958,C1QA
210593,GO:0001682,RPP40
210594,GO:0061640,DCTN3
210595,GO:0045892,NELFCD


In [24]:
returned_all=CA.hypergeometric_test_for_all_communities(reference_network=HIPPIE_nx,
                                           prior_knowledge_df=GOA_bio_proc_df,
                                           prior_knowledge_on="GO ID", 
                                           name_on="DB Object Symbol")

In [25]:
returned_all 

,Community_name,GO ID,p-value,Erichment_Score,Genes in Module,Intersecting Genes,The number of intersecting genes,Process_Gene,The number of components of prior_knowledge
0,Module_2,GO:0000122,2.198714e-03,3.785060,"[DISC1, SFPQ, REL, UBE2D3, GPS2, TRIM68]","[SFPQ, UBE2D3, GPS2]",3,"[LINC-PINT, E2F8, FEZF1, CNOT1, NUPR2, HELT, N...",968
1,Module_2,GO:0045087,5.068671e-04,4.566379,"[DISC1, SFPQ, REL, UBE2D3, GPS2, TRIM68]","[SFPQ, TRIM68, REL]",3,"[TRGV2, DEFB131B, TRGV5, TRGV8, TRGV4, TRDV1, ...",585
2,Module_2,GO:0045944,4.076125e-03,3.441766,"[DISC1, SFPQ, REL, UBE2D3, GPS2, TRIM68]","[SFPQ, REL, GPS2]",3,"[E2F8, ZBTB7C, NKX2-6, ZNF840P, HELT, MSGN1, N...",1200
3,Module_3,GO:0000122,3.705554e-03,3.443890,"[ZMYND10, CALM1, UBE2I, HIC1, TCF4, OGT, NEUROD1]","[OGT, UBE2I, HIC1]",3,"[LINC-PINT, E2F8, FEZF1, CNOT1, NUPR2, HELT, N...",968
4,Module_3,GO:0006357,1.633251e-03,3.305400,"[ZMYND10, CALM1, UBE2I, HIC1, TCF4, OGT, NEUROD1]","[OGT, NEUROD1, TCF4, HIC1]",4,"[MT-RNR1, EPOP, UNCX, ZNF98, ZNF716, ZNF724, Z...",1697
...,...,...,...,...,...,...,...,...,...
277,Module_33,GO:0008360,3.730783e-05,5.776614,"[HDAC4, ELAVL1, SENP1, RAC2, UBD, RAC3, CDC42,...","[RAC2, RAC3, RHOD]",3,"[PALM3, PALM2AKAP2, CCL24, LST1, CDC42EP2, SLC...",151
278,Module_33,GO:0010592,2.493232e-07,8.263737,"[HDAC4, ELAVL1, SENP1, RAC2, UBD, RAC3, CDC42,...","[RAC2, CDC42, HDAC4]",3,"[MSTN, ARPC2, AVIL, WNT1, RAC2, PIK3R1, PIK3CA...",29
279,Module_33,GO:0016567,1.433593e-03,3.902231,"[HDAC4, ELAVL1, SENP1, RAC2, UBD, RAC3, CDC42,...","[UBD, ARRB2, CDC42]",3,"[TMEM129, UBA6, ASB14, RFPL4A, MARCHF11, ZSWIM...",522
280,Module_33,GO:0030865,9.091819e-08,8.772185,"[HDAC4, ELAVL1, SENP1, RAC2, UBD, RAC3, CDC42,...","[RAC3, RHOD, RAC2]",3,"[WDR1, EPB41L3, RHOH, RHOG, RHOF, U3KQA9, RHOA...",21


In [26]:
returned_all.to_csv(f'../Outputs/AndrogenReceptor_inferred_csn_overrepresentation_results.tab',sep="\t", index=False)